# 第2回講義 宿題

## 課題
今回のLessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) をソフトマックス回帰によって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 80%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください．**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)．
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Homeworkタブから`lecture02`を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Homeworkタブから`lecture02 (code)`を選択して**提出してください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します．（採点スケジュールは別アナウンス）
- 締切後の点数を最終的な評価とします．

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### データの読み込み（このセルは修正しないでください）

In [ ]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/x_train.npy')
    y_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

In [ ]:
x_train, y_train, x_test = load_fashionmnist()

### ソフトマックス回帰の実装

In [ ]:
def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=1e+10))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def softmax(x):
    x -= np.max(x)
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

# weights
W = np.random.normal(0, 0.1, (784, 10))
b = np.random.normal(0, 0.1, (10,))

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

def train(x, y, eps=1.0):
    global W, b
    batch_size = x.shape[0]
    y_hat = softmax(np.dot(x, W) + b)
    cost = np.sum(-y * np.log(y_hat))
    
    delta = y_hat - y
    dW = np.dot(x.T, delta)/batch_size
    db = np.sum(delta, axis=0)/batch_size
    
    W -= eps * dW
    b -= eps * db

def valid(x, y):
    y_hat = softmax(np.dot(x, W) + b)
    y_pred = np.argmax(y_hat, axis=1)
    y_true = np.argmax(y, axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

for epoch in range(1):
    # オンライン学習
    for i in range(len(x_train)):
        train(x_train[i:i+1], y_train[i:i+1], eps=0.01)

    train = valid(x_train, y_train)
    valid = valid(x_valid, y_valid)
    print('epoch:', epoch+1, ', accuracy(train):', train, ', accuracy(valid):', valid)

y_pred = np.argmax(softmax(np.dot(x_test, W) + b), axis=1)
submission = pd.Series(y_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/submission_pred.csv', header=True, index_label='id')

epoch: 1 , accuracy(train): 0.8412962962962963 , accuracy(valid): 0.827
